# Group project

### Import libraries and data

In [1]:
# Library imports
import time, math, cv2, imutils, os, json, random
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Task 1 data imports

# get_data: Import data from file system
# base_path: Path to the containing folder
# clip_indexes: Array of clip index values (0-40)
# Returns array of form [[json, [img1, ...]], [json, [img1, ...]], ...]
def get_data(base_path, clip_indexes):
    img_data = []
    for subdir, dirs, files in os.walk(base_path):
        # Assumes json is opened first, then all images for it in order
        for file in files:
            if (file == "annotation.json"):
                with open(os.path.join(subdir, file)) as f:
                    img_data += [[json.load(f), []]]
            if (len(file) == 7 and file[-4:] == ".jpg" and int(file[:3]) in clip_indexes):
                img_data[-1][1] += [cv2.imread(os.path.join(subdir, file), cv2.IMREAD_GRAYSCALE)]
    return img_data

t0 = time.time()

print("Loading data...")

# Load data as [annotation, bgr image] pairs
train_data = get_data("data/train/clips", [40])
test_data = get_data("data/test/clips", [40])

print("Done.")
print("Image load time:", time.time() - t0)

print("Training clips:", len(train_data))
print("Testing clips:", len(test_data))

Loading data...
Done.
Image load time: 18.641541957855225
[[{'velocity': [-1.2287469463, -0.0101401592], 'bbox': {'top': 340.3418273926, 'right': 868.1537475586, 'bottom': 393.3275146484, 'left': 784.6638793945}, 'position': [21.30970213, 2.7243980956]}], [array([[146, 143, 170, ..., 153, 154, 154],
       [149, 145, 163, ..., 155, 157, 158],
       [147, 144, 152, ..., 155, 155, 156],
       ...,
       [ 22,  22,  22, ...,  11,  11,  11],
       [ 21,  21,  21, ...,  13,  12,  12],
       [ 20,  20,  20, ...,  13,  13,  13]], dtype=uint8), array([[146, 153, 166, ..., 156, 156, 156],
       [150, 154, 162, ..., 156, 156, 156],
       [146, 146, 150, ..., 156, 157, 157],
       ...,
       [ 24,  23,  23, ...,  12,  12,  12],
       [ 23,  23,  22, ...,  12,  12,  11],
       [ 22,  22,  22, ...,  12,  11,  11]], dtype=uint8)]]
Training clips: 1074
Testing clips: 269


## Task 1

In [ ]:
# TODO: Car detection